In [88]:
import censusdata
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
import geopandas as gpd

In [149]:
#censusdata.search('acs5', 2015, 'concept', 'age')

In [90]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B01001A'))

In [137]:
df = censusdata.download('acs5', 2015, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001_001E', 'B01001_002E', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E',
                              'B01001_011E','B01001_012E', 'B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_016E', 
                              'B01001_017E','B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_026E', 
                              'B01001_027E','B01001_028E', 'B01001_029E', 'B01001_030E', 'B01001_031E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B01001_001E' : 'Total', 'B01001_002E' : 'Male', 'B01001_007E' : 'M 18-19', 'B01001_008E' : 'M 20-24', 
                             'B01001_009E' : 'M 25-29', 'B01001_010E' : 'M 30-34','B01001_011E' : 'M 35-44','B01001_012E' : 'M 45-54', 
                             'B01001_013E' : 'M 55-64', 'B01001_014E' : 'M 65-74', 'B01001_015E' : 'M 75-84', 
                             'B01001_016E' : 'M 85+', 'B01001_017E' : 'Female', 'B01001_022E' : 'F 18-19', 'B01001_023E' : 'F 20-24', 
                             'B01001_024E' : 'F 25-29', 'B01001_025E' : 'F 30-34', 'B01001_026E' : 'F 35-44', 'B01001_027E' : 'F 45-54',
                             'B01001_028E' : 'F 55-64', 'B01001_029E' : 'F 65-74', 'B01001_030E' : 'F 75-84', 'B01001_031E' : 'F 85+'}, 
                              inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

df

,TRACT,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 18-19,F 20-24,F 25-29,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 85+
0,20303,3574,2059,35,30,63,129,331,161,255,...,28,4,19,19,1515,112,10,94,0,12
1,40401,2523,962,29,0,0,31,146,132,66,...,30,0,0,7,1561,155,119,50,0,7
2,40801,4098,1803,42,0,37,69,137,409,61,...,38,13,19,0,2295,176,88,82,39,10
3,50101,5487,2857,21,91,37,121,236,302,250,...,70,25,24,45,2630,322,112,110,30,41
4,50901,4740,2855,23,98,54,169,259,410,306,...,25,29,22,0,1885,101,205,46,22,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,101101,3808,1866,26,0,60,114,165,108,219,...,83,50,8,8,1942,137,124,87,51,33
200,130300,4680,2197,38,8,6,48,83,195,163,...,63,32,10,45,2483,164,106,179,110,19
201,140102,4700,2127,6,32,10,152,219,73,150,...,81,96,27,37,2573,211,92,82,75,84
202,140300,6830,3409,129,53,40,178,299,214,205,...,134,17,104,123,3421,122,291,307,55,48


In [138]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

In [139]:
prct_age

,WARD_PRECI,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 18-19,F 20-24,F 25-29,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 85+
0,0102,2671.0,1473.0,27.0,28.0,10.0,100.0,197.0,240.0,101.0,...,30.0,39.0,27.0,1.0,1198.0,114.0,71.0,92.0,34.0,19.0
1,0105,3807.0,2271.0,29.0,58.0,69.0,78.0,314.0,250.0,192.0,...,44.0,8.0,0.0,15.0,1536.0,49.0,174.0,96.0,106.0,5.0
2,0203,1440.0,638.0,1.0,4.0,3.0,11.0,75.0,78.0,66.0,...,19.0,12.0,4.0,6.0,802.0,79.0,46.0,30.0,14.0,4.0
3,0205,1549.0,695.0,0.0,0.0,0.0,17.0,91.0,100.0,92.0,...,17.0,12.0,2.0,0.0,853.0,97.0,50.0,30.0,6.0,0.0
4,0204,1788.0,804.0,0.0,5.0,1.0,20.0,94.0,108.0,103.0,...,18.0,14.0,2.0,1.0,984.0,121.0,67.0,38.0,7.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,1818,3229.0,1586.0,53.0,28.0,17.0,85.0,117.0,96.0,98.0,...,47.0,24.0,35.0,37.0,1644.0,81.0,112.0,121.0,29.0,24.0
250,0502A,1428.0,681.0,64.0,48.0,17.0,64.0,152.0,126.0,71.0,...,0.0,0.0,4.0,4.0,748.0,26.0,0.0,6.0,5.0,42.0
251,0115,71.0,34.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,...,2.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,1.0
252,0104,2075.0,1144.0,10.0,24.0,30.0,72.0,121.0,134.0,135.0,...,12.0,8.0,6.0,8.0,931.0,39.0,59.0,51.0,45.0,12.0


In [140]:
'0113' in prct_age['WARD_PRECI'].unique()

False

In [141]:
prct_age.to_csv('Precinct_Age_Distribution_Based_On_2011-15_ACS_Census_Data.csv')

In [142]:
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)
        
sum_column = neigh_age_11_15['F 55-64'] + neigh_age_11_15['F 65-74'] + neigh_age_11_15['F 75-84'] + neigh_age_11_15['F 85+']
neigh_age_11_15.insert(23, 'F 55+', sum_column)

sum_column = neigh_age_11_15['M 55-64'] + neigh_age_11_15['M 65-74'] + neigh_age_11_15['M 75-84'] + neigh_age_11_15['M 85+']
neigh_age_11_15.insert(12, 'M 55+', sum_column)

sum_column = neigh_age_11_15['F 55+'] + neigh_age_11_15['M 55+']
neigh_age_11_15['Total 55+'] = sum_column

sum_column = neigh_age_11_15['F 18-19'] + neigh_age_11_15['F 20-24'] + neigh_age_11_15['F 25-29'] + \
             neigh_age_11_15['M 18-19'] + neigh_age_11_15['M 20-24'] + neigh_age_11_15['M 25-29']
neigh_age_11_15['Total 18-29'] = sum_column


cols = list(neigh_age_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_11_15)):
        neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)

neigh_age_11_15

,NEIGHBORHOOD,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 55+,F 85+,Total 55+,Total 18-29
0,Roslindale,29018,0.461,0.008,0.006,0.008,0.012,0.039,0.039,0.037,...,0.005,0.539,0.032,0.027,0.022,0.020,0.079,0.009,0.220,0.045
1,Jamaica Plain,42757,0.466,0.009,0.009,0.011,0.024,0.066,0.051,0.038,...,0.004,0.534,0.026,0.021,0.018,0.019,0.069,0.011,0.195,0.051
2,Mission Hill,13921,0.509,0.045,0.036,0.035,0.093,0.071,0.026,0.027,...,0.004,0.490,0.015,0.018,0.013,0.011,0.090,0.048,0.164,0.139
3,Longwood,6240,0.387,0.100,0.048,0.034,0.056,0.037,0.012,0.014,...,0.002,0.613,0.005,0.020,0.007,0.007,0.227,0.193,0.270,0.191
4,Bay Village,1398,0.476,0.021,0.008,0.006,0.014,0.054,0.069,0.024,...,0.019,0.525,0.016,0.014,0.009,0.006,0.074,0.045,0.172,0.075
5,Leather District,537,0.467,0.048,0.020,0.013,0.020,0.054,0.035,0.026,...,0.009,0.533,0.019,0.009,0.004,0.007,0.115,0.095,0.214,0.112
6,Chinatown,2718,0.468,0.046,0.019,0.013,0.019,0.053,0.040,0.025,...,0.011,0.532,0.018,0.010,0.005,0.008,0.113,0.090,0.209,0.109
7,North End,6962,0.516,0.003,0.005,0.009,0.048,0.146,0.078,0.039,...,0.008,0.484,0.010,0.006,0.005,0.000,0.017,0.005,0.131,0.056
8,Roxbury,50606,0.474,0.029,0.017,0.016,0.035,0.044,0.027,0.023,...,0.003,0.526,0.028,0.030,0.026,0.023,0.107,0.029,0.210,0.083
9,South End,23662,0.514,0.010,0.008,0.011,0.029,0.067,0.066,0.032,...,0.010,0.486,0.025,0.014,0.017,0.006,0.054,0.016,0.194,0.057


In [143]:
neigh_age_11_15.to_csv('Boston_Neighborhood_Estimated_Age_Distribution_ACS_2011-2015.csv')

In [152]:
df2 = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001_001E', 'B01001_002E', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E',
                              'B01001_011E','B01001_012E', 'B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_016E', 
                              'B01001_017E','B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_026E', 
                              'B01001_027E','B01001_028E', 'B01001_029E', 'B01001_030E', 'B01001_031E'])
df2 = df2.reset_index(drop=False)
df2.rename(columns = {'index' : 'TRACT', 'B01001_001E' : 'Total', 'B01001_002E' : 'Male', 'B01001_007E' : 'M 18-19', 'B01001_008E' : 'M 20-24', 
                             'B01001_009E' : 'M 25-29', 'B01001_010E' : 'M 30-34','B01001_011E' : 'M 35-44','B01001_012E' : 'M 45-54', 
                             'B01001_013E' : 'M 55-64', 'B01001_014E' : 'M 65-74', 'B01001_015E' : 'M 75-84', 
                             'B01001_016E' : 'M 85+', 'B01001_017E' : 'Female', 'B01001_022E' : 'F 18-19', 'B01001_023E' : 'F 20-24', 
                             'B01001_024E' : 'F 25-29', 'B01001_025E' : 'F 30-34', 'B01001_026E' : 'F 35-44', 'B01001_027E' : 'F 45-54',
                             'B01001_028E' : 'F 55-64', 'B01001_029E' : 'F 65-74', 'B01001_030E' : 'F 75-84', 'B01001_031E' : 'F 85+'}, 
                              inplace = True)
for x in range(len(df2)):
    df2.loc[x, 'TRACT'] = df2['TRACT'].unique()[x].geo[2][1]
    
df2 = df2.astype({"TRACT": int})

df2

,TRACT,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 18-19,F 20-24,F 25-29,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 85+
0,802,6701,3778,100,316,108,591,979,549,226,...,36,29,72,35,2923,41,39,97,103,124
1,10600,3168,1299,0,29,0,18,182,225,156,...,85,40,20,0,1869,43,12,0,0,34
2,40100,2442,1118,9,0,9,0,145,166,105,...,34,20,35,8,1324,82,27,0,8,0
3,40300,4665,2039,10,0,0,32,273,380,222,...,39,33,18,0,2626,364,233,56,0,21
4,50400,2284,1325,14,0,0,52,168,201,140,...,25,0,0,45,959,34,28,32,10,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,82100,5998,2659,55,49,38,29,360,355,84,...,72,74,144,81,3339,140,144,242,117,16
200,60200,2103,1014,0,5,5,42,267,184,60,...,23,15,6,12,1089,54,41,0,20,0
201,90400,4396,2013,127,112,32,75,178,63,127,...,28,12,52,0,2383,99,151,140,113,83
202,120500,2850,1449,17,6,0,94,301,233,105,...,14,38,0,25,1401,90,9,59,0,31


In [153]:
prct_age2 = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df2.columns[1:])

prct_age2.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age2)):
    prct = prct_age2['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df2['TRACT'] == temp.loc[n, 'TRACT']
            for c in df2.columns[1:]:
                prct_age2[c].iloc[i] += round(df2[filt][c] * dec)

In [154]:
prct_age2.to_csv('Precinct_Age_Distribution_Based_On_2015-19_ACS_Census_Data.csv')

In [155]:
neigh_age_15_19 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age2.columns[1:])
neigh_age_15_19.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_15_19)):
    neighb = neigh_age_15_19['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age2['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age2.columns[1:]:
                try:
                    neigh_age_15_19[c].iloc[i] += round(prct_age2[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_15_19.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_15_19[c] = neigh_age_15_19[c].round().astype(int)
        
sum_column = neigh_age_15_19['F 55-64'] + neigh_age_15_19['F 65-74'] + neigh_age_15_19['F 75-84'] + neigh_age_15_19['F 85+']
neigh_age_15_19.insert(23, 'F 55+', sum_column)

sum_column = neigh_age_15_19['M 55-64'] + neigh_age_15_19['M 65-74'] + neigh_age_15_19['M 75-84'] + neigh_age_15_19['M 85+']
neigh_age_15_19.insert(12, 'M 55+', sum_column)

sum_column = neigh_age_15_19['F 55+'] + neigh_age_15_19['M 55+']
neigh_age_15_19['Total 55+'] = sum_column

sum_column = neigh_age_15_19['F 18-19'] + neigh_age_15_19['F 20-24'] + neigh_age_15_19['F 25-29'] + \
             neigh_age_15_19['M 18-19'] + neigh_age_15_19['M 20-24'] + neigh_age_15_19['M 25-29']
neigh_age_15_19['Total 18-29'] = sum_column

cols = list(neigh_age_15_19.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_15_19)):
        neigh_age_15_19.loc[i, c] = round(neigh_age_15_19.loc[i, c]/neigh_age_15_19.loc[i, 'Total'], 3)
        


neigh_age_15_19

,NEIGHBORHOOD,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 55+,F 85+,Total 55+,Total 18-29
0,Roslindale,30205,0.467,0.007,0.005,0.006,0.012,0.043,0.036,0.043,...,0.008,0.533,0.033,0.025,0.025,0.019,0.080,0.010,0.222,0.042
1,Jamaica Plain,43866,0.462,0.007,0.003,0.007,0.019,0.060,0.064,0.041,...,0.005,0.538,0.031,0.018,0.016,0.018,0.062,0.010,0.192,0.041
2,Mission Hill,14248,0.517,0.046,0.033,0.050,0.091,0.078,0.031,0.023,...,0.004,0.484,0.010,0.009,0.008,0.009,0.079,0.053,0.143,0.151
3,Longwood,6549,0.420,0.111,0.051,0.041,0.052,0.048,0.015,0.007,...,0.004,0.579,0.002,0.004,0.002,0.012,0.233,0.215,0.266,0.209
4,Bay Village,1465,0.472,0.027,0.009,0.003,0.016,0.041,0.072,0.027,...,0.011,0.528,0.026,0.007,0.007,0.008,0.052,0.031,0.174,0.079
5,Leather District,621,0.462,0.061,0.013,0.011,0.023,0.056,0.047,0.021,...,0.006,0.538,0.013,0.016,0.008,0.003,0.108,0.081,0.196,0.114
6,Chinatown,3113,0.463,0.058,0.013,0.010,0.022,0.056,0.049,0.022,...,0.007,0.536,0.014,0.016,0.008,0.004,0.103,0.075,0.194,0.111
7,North End,6784,0.505,0.003,0.003,0.009,0.041,0.152,0.090,0.034,...,0.002,0.495,0.009,0.011,0.007,0.000,0.022,0.004,0.104,0.049
8,Roxbury,53772,0.462,0.026,0.011,0.013,0.034,0.043,0.034,0.021,...,0.004,0.538,0.027,0.028,0.032,0.020,0.104,0.025,0.199,0.078
9,South End,24202,0.477,0.010,0.004,0.005,0.018,0.061,0.062,0.036,...,0.008,0.523,0.020,0.017,0.024,0.008,0.065,0.016,0.204,0.050


In [156]:
neigh_age_15_19.to_csv('Boston_Neighborhood_Estimated_Age_Distribution_ACS_2015-2019.csv')

In [150]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B01001'))

In [151]:
# A - White -
# B - African American -
# C - Native American
# D - Asian

In [174]:
df = censusdata.download('acs5', 2015, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001D_001E', 'B01001D_002E', 'B01001D_007E', 'B01001D_008E', 'B01001D_009E', 'B01001D_010E',
                              'B01001D_011E','B01001D_012E', 'B01001D_013E', 'B01001D_014E', 'B01001D_015E', 'B01001D_016E', 
                              'B01001D_017E','B01001D_022E', 'B01001D_023E', 'B01001D_024E', 'B01001D_025E', 'B01001D_026E', 
                              'B01001D_027E','B01001D_028E', 'B01001D_029E', 'B01001D_030E', 'B01001D_031E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B01001D_001E' : 'Total', 'B01001D_002E' : 'Male', 'B01001D_007E' : 'M 18-19', 'B01001D_008E' : 'M 20-24', 
                             'B01001D_009E' : 'M 25-29', 'B01001D_010E' : 'M 30-34','B01001D_011E' : 'M 35-44','B01001D_012E' : 'M 45-54', 
                             'B01001D_013E' : 'M 55-64', 'B01001D_014E' : 'M 65-74', 'B01001D_015E' : 'M 75-84', 
                             'B01001D_016E' : 'M 85+', 'B01001D_017E' : 'Female', 'B01001D_022E' : 'F 18-19', 'B01001D_023E' : 'F 20-24', 
                             'B01001D_024E' : 'F 25-29', 'B01001D_025E' : 'F 30-34', 'B01001D_026E' : 'F 35-44', 'B01001D_027E' : 'F 45-54',
                             'B01001D_028E' : 'F 55-64', 'B01001D_029E' : 'F 65-74', 'B01001D_030E' : 'F 75-84', 'B01001D_031E' : 'F 85+'}, 
                              inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

prct_age.to_csv('Precinct_ASIAN_Age_Distribution_Based_On_2011-15_ACS_Census_Data.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)

cols = list(neigh_age_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_11_15)):
        neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)

neigh_age_11_15

,NEIGHBORHOOD,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 18-19,F 20-24,F 25-29,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 85+
0,Roslindale,590,0.371,0.007,0.032,0.056,0.010,0.025,0.122,0.036,...,0.000,0.066,0.088,0.029,0.090,0.142,0.041,0.073,0.002,0.010
1,Jamaica Plain,2336,0.434,0.012,0.038,0.046,0.084,0.102,0.044,0.031,...,0.025,0.047,0.104,0.114,0.097,0.045,0.038,0.018,0.010,0.018
2,Mission Hill,2096,0.495,0.020,0.160,0.102,0.039,0.063,0.026,0.027,...,0.042,0.168,0.060,0.045,0.016,0.028,0.042,0.028,0.017,0.013
3,Longwood,972,0.359,0.029,0.120,0.048,0.025,0.044,0.036,0.036,...,0.163,0.151,0.051,0.033,0.015,0.051,0.039,0.021,0.026,0.015
4,Bay Village,452,0.451,0.009,0.024,0.040,0.015,0.027,0.084,0.073,...,0.013,0.024,0.046,0.020,0.066,0.095,0.084,0.044,0.027,0.055
5,Leather District,262,0.462,0.011,0.042,0.046,0.023,0.031,0.065,0.076,...,0.019,0.034,0.061,0.027,0.050,0.069,0.099,0.046,0.034,0.046
6,Chinatown,1280,0.462,0.010,0.041,0.046,0.023,0.030,0.066,0.078,...,0.019,0.032,0.061,0.026,0.051,0.072,0.098,0.045,0.035,0.047
7,North End,255,0.463,0.004,0.047,0.208,0.082,0.043,0.039,0.016,...,0.008,0.106,0.078,0.114,0.118,0.020,0.035,0.000,0.000,0.000
8,Roxbury,1922,0.507,0.057,0.148,0.075,0.042,0.040,0.041,0.031,...,0.123,0.118,0.057,0.052,0.040,0.014,0.024,0.011,0.005,0.002
9,South End,4065,0.466,0.010,0.046,0.050,0.035,0.044,0.065,0.049,...,0.011,0.031,0.080,0.058,0.075,0.067,0.052,0.046,0.024,0.027


In [175]:
neigh_age_11_15.to_csv('Boston_Neighborhood_Estimated_ASIAN_Age_Distribution_ACS_2011-2015.csv')